In [49]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!ls /content/drive/MyDrive/
!pip install --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib


 02_Lexical_Analysis1.gdoc
'1200108 (1).c.gdoc'
 1200108.c.gdoc
'2022-04-28 22-02-56.mp4'
'2022-05-04 20-08-13.mp4'
'90 MCQ To introEs.gdoc'
'Adobe Scan 18 Nov 2023.pdf'
'Adobe Scan 26 Jun 2023 (1)_1.jpg'
'Adobe Scan 26 Jun 2023 (1)_2.jpg'
'Ahmed_Yasser_cv (1).pdf'
 Ahmed_Yasser_cv.pdf
 AhmedYasserHussein_1200108_A1.gdoc
 AhmedYasserHussein_1200108_Assignment_2_lecture_8.gdoc
 AhmedYasserHussein_1200108_SummerFees.jpeg
 AhmedYasserHussein_CoverLetter.pdf
'AhmedYasserHussein_CV (1).pdf'
 AhmedYasserHussein_CV.pdf
 Ahmed_Yasser_Hussein_final_lab.rar
 AhmedYasserHussein_FinalProject.rar
'Ahmed Yasser Hussein.gdoc'
 Ahmed_Yasser_Hussein_lab_1.rar
 Ahmed_Yasser_Hussein_lab_2.rar
 Ahmed_Yasser_Hussein_lab_3.rar
 AhmedYasserHussein_Miniproject3.rar
 AhmedYasserHussein_Miniproject4.rar
 AhmedYasserHussein_project1200108.pdf
'AhmedYasserHussein_References list(Unix) .gdoc'
'AhmedYasserHussein_Resume (10).pdf'
'AhmedYasserHussein_resume (1) (1).pdf'
'AhmedYasserHussein_Resume (11).pdf'
'AhmedYas

In [51]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
import os
import io

# Authenticate and create the PyDrive client
auth.authenticate_user()

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.auth import default

creds, _ = default()

service = build('drive', 'v3', credentials=creds)

# Function to list files in a folder
def list_files_in_folder(folder_id, max_files=10):
    query = f"'{folder_id}' in parents and mimeType != 'application/vnd.google-apps.folder'"
    results = service.files().list(
        q=query,
        pageSize=max_files,
        fields="files(id, name)"
    ).execute()
    return results.get('files', [])

# Function to download files
def download_file(file_id, file_name, destination_folder):
    request = service.files().get_media(fileId=file_id)
    file_path = os.path.join(destination_folder, file_name)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Downloaded {file_name}: {int(status.progress() * 100)}%")
    return file_path

# Main logic
def download_first_n_files_in_each_folder(folder_id, output_dir, max_files=10):
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.folder'"
    subfolders = service.files().list(q=query, fields="files(id, name)").execute().get('files', [])

    for subfolder in subfolders:
        subfolder_id = subfolder['id']
        subfolder_name = subfolder['name']
        print(f"Processing folder: {subfolder_name}")

        # Create subfolder in output directory
        subfolder_path = os.path.join(output_dir, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)

        # List and download first N files
        files = list_files_in_folder(subfolder_id, max_files)
        for file in files:
            download_file(file['id'], file['name'], subfolder_path)

# Replace with your folder ID
root_folder_id = "1OVGA3CGnEKjyg_k_L8MP2RO5R3oDIbHE"  # Google Drive folder ID
output_directory = "/content/dataset"
os.makedirs(output_directory, exist_ok=True)

download_first_n_files_in_each_folder(root_folder_id, output_directory, max_files=20)


Processing folder: 16_024
Downloaded 16(85).png: 100%
Downloaded 16(843).png: 100%
Downloaded 16(88).png: 100%
Downloaded 16(879).png: 100%
Downloaded 16(909).png: 100%
Downloaded 16(797).png: 100%
Downloaded 16(913).png: 100%
Downloaded 16(895).png: 100%
Downloaded 16(850).png: 100%
Downloaded 16(958).png: 100%
Downloaded 16(904).png: 100%
Downloaded 16(957).png: 100%
Downloaded 16(7).png: 100%
Downloaded 16(929).png: 100%
Downloaded 16(755).png: 100%
Downloaded 16(824).png: 100%
Downloaded 16(760).png: 100%
Downloaded 16(901).png: 100%
Downloaded 16(816).png: 100%
Downloaded 16(876).png: 100%
Processing folder: bar_110
Downloaded MUSCIMA-pp_2.0___CVC-MUSCIMA_W-23_N-13_D-ideal___619.png: 100%
Downloaded MUSCIMA-pp_2.0___CVC-MUSCIMA_W-23_N-06_D-ideal___778.png: 100%
Downloaded MUSCIMA-pp_2.0___CVC-MUSCIMA_W-22_N-14_D-ideal___555.png: 100%
Downloaded MUSCIMA-pp_2.0___CVC-MUSCIMA_W-23_N-13_D-ideal___431.png: 100%
Downloaded MUSCIMA-pp_2.0___CVC-MUSCIMA_W-23_N-06_D-ideal___646.png: 100%
D

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import skimage as sk
import numpy as np
import os
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [54]:
def hu_moments(img):
    '''
    Hu Moments ( or rather Hu moment invariants ) are a set of 7 numbers calculated using central
    moments that are invariant to image transformations. The first 6 moments have been proved to be
    invariant to translation,scale, and rotation, and reflection. While the 7th moment’s sign
    changes for image reflection.

    img: binary image, pixel values 0:1
    return: 7 Hu moments
    '''
    mu = sk.measure.moments_central(img)
    nu = sk.measure.moments_normalized(mu)
    moments = sk.measure.moments_hu(nu)
    return moments

In [68]:




def feature_extraction(img):

    if len(img.shape) > 2:
        img = sk.color.rgb2gray(img)


    img_resized = sk.transform.resize(img, (64, 64), anti_aliasing=False )


    img_bin = (img_resized > 0.5).astype(np.uint8)


    hog_features = sk.feature.hog(
        img_bin,
        orientations=8,  # Increased for more detailed features
        pixels_per_cell=(4, 4),  # Adjusted for typical HOG settings
        cells_per_block=(2, 2),
        block_norm='L1',
        feature_vector=True  # Ensure a 1D feature vector
    )
    hu_moment=hu_moments(img_bin)
    hog_features=np.concatenate((hog_features,hu_moment))
    return hog_features

# When preparing your dataset



In [69]:
# Load images and labels
def load_dataset(folder_path):
    X = []  # Features
    y = []  # Labels
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Add other formats if needed
                    image = imread(file_path)
                    if(image.ndim!=2 ):
                      continue
                    features = feature_extraction(image)
                    X.append(features)
                    y.append(class_name)  # Folder name is the label
    return X, y

dataset_path = "/content/dataset"  # Path to your dataset
X, y = load_dataset(dataset_path)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

import joblib
joblib.dump(knn, "knn_model.pkl")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

               precision    recall  f1-score   support

            #       0.94      1.00      0.97        16
          #_2       0.50      0.50      0.50         2
      #_b_002       0.43      1.00      0.60         3
      #_b_005       0.50      0.33      0.40         3
      #_b_007       0.67      0.29      0.40         7
       16_021       0.67      0.50      0.57         4
       16_022       0.50      0.67      0.57         3
       16_023       0.60      0.75      0.67         4
       16_024       0.75      0.75      0.75         4
       16_025       0.75      1.00      0.86         6
   16_b_n_026       0.50      0.33      0.40         3
   16_b_n_029       0.60      0.75      0.67         4
   16_b_r_031       1.00      0.75      0.86         4
   16_b_r_032       0.00      0.00      0.00         0
   16_b_r_034       1.00      0.25      0.40         4
        1_001       0.43      0.75      0.55         4
        1_002       1.00      0.40      0.57         5
        1

['knn_model.pkl']